# Chapter 7. 함수 데코레이터와 클로저

## 7.1 데커레이터 기본 지식
데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

예시처럼 decoreate라는 이름의 데커레이터가 있다면
    
```python
@decorate
def target():
    print('running target()')
```
위 코드는 다음과 같이 작동한다
```python
def target():
    print('running target()')

target = decorate(target)
```
두 가지 코드는 동일하다. 두 코드를 실행하면 target은 원래의 target() 함수를 가리키는 것이 아니고 decorate(target)이 반환한 함수를 가리키게 된다. 

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [8]:
target()    # deco함수의 inner()가 실행된다.

running inner()


In [7]:
target      # deco함수의 inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점


In [14]:
# example 7-2. register.py
registry = []

def register(func):
    print(f'running register({func}!!)')      # 데코레이터가 실행될 때마다 실행된다.
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x105f00ca0>!!)
running register(<function f2 at 0x105de08b0>!!)


In [4]:
main()

running main()
registry -> [<function f1 at 0x106751dc0>, <function f2 at 0x106751e50>]
running f1()
running f2()
running f3()


In [12]:
!python Ch7/register.py

running register(<function f1 at 0x1053e8160>)
running register(<function f2 at 0x1053e81f0>)
running main()
registry -> [<function f1 at 0x1053e8160>, <function f2 at 0x1053e81f0>]
running f1()
running f2()
running f3()


In [16]:
from Ch7 import register

register.registry

[<function Ch7.register.f1()>, <function Ch7.register.f2()>]

## 7.3 데커레이터로 개선한 전략 패턴

In [22]:
# example 7-3. promotion 데커레이터로 채운 promos 리스트

promos = []

def promotion(anything):
    promos.append(anything)
    return anything

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 5% 할인"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 구매 상품의 7% 할인"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인해 줄 금액을 반환한다."""
    return max(promo(order) for promo in promos)


In [23]:
promos

[<function __main__.fidelity(order)>,
 <function __main__.bulk_item(order)>,
 <function __main__.large_order(order)>]

## 7.4 변수 범위 규칙

In [24]:
# 지역 및 전역 변수를 읽는 함수
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [26]:
# 지역변수가 없으면 전역변수를 참조한다.
b = 6
f1(3)

3
6


In [32]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [35]:
# 지역변수가 있지만 전역변수를 가져오고 싶다면 global 키워드를 사용한다.
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [39]:
# BYTE CODE
from dis import dis

print(dis(f1),end='========================================\n')
print(dis(f2),end='========================================\n')
print(dis(f3),end='========================================\n')

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None========================================
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None====================================

## 7.5 클로저(Closure)
클로져 != 익명함수
클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonlocal)변수를 포함한 확장 범위를 가진 함수다. 익명 함수여부는 중요하지 않고, 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다.  
아래의 예제에서 `avg()`함수가 점차 증가하는 일련의 값의 평균을 계산한다고 가정했을 때, `avg()`함수는 `series`변수를 참조한다. `series`변수는 `avg()`함수의 본체에서 정의되지 않았다. 따라서 `avg()`함수는 `series`변수를 포함한 확장 범위를 가진다. 이런 함수를 클로저라고 한다.

```python
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```


avg()는 이전 값을 어떻게 기억하고 있는 것인지 클래스를 이용해서 구현해서 살펴봅니다

In [40]:
class Averager:
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [3]:
avg = Averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [4]:
def make_averager():
    series = []
    
    # ----클로저 -----
    def averager(new_value):
        series.append(new_value)        # serise는 자유 변수
        total = sum(series)
        return total / len(series)

    return averager

In [18]:
avg = make_averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


`Averager` class의 a`vg()`함수가 데이터를 보관하는 방법은 `self.series` 에 저장된다. 하지만 클로저의 `avg()`함수는 어디서 찾을 수 있을까?
`make_averager()` 함수안에서 series = []로 초기화하고 있으므로 `series`는 해당 함수의 지역 변수이다. 하지만 `avg(10)`을 호출할 때, make_average()함수는 이미 반환했으므로 지역 범위도 이미 사라졌다.
그렇기 때문에 average 안에 있는 **series는 자유변수(free variable)** 이다. 자유 변수는 지역 범위에 바인딩되어 있지 않은 변수를 의미한다.

In [27]:
print(avg.__code__.co_varnames)     # (지역)변수이름
print(avg.__code__.co_freevars)     # 자유변수

('new_value', 'total')
('series',)


In [26]:
# series에 대한 바인딩은 반환된 avg()함수의 __closure__ 속성에 저장된다.
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents) # 각 항목은 co freevars이름에 대응된다.


('series',)
(<cell at 0x106f3ca90: list object at 0x107cd0cc0>,)
[10, 11, 12]


## 7.6 nonlocal 선언
앞에서 구현한 make_averager()는 효율적이지 않다. 잘못된 코드에서 잘 못된 부분을 찾아보자

In [29]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [30]:
avg = make_averager()
print(avg(10))      # closure에서 count와 total을 사용할 수 없다.

UnboundLocalError: local variable 'count' referenced before assignment

In [31]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [32]:
avg = make_averager()
print(avg(10))

10.0


## 7.7 간단한 데코레이터 구현하기
데코레이터된 함수를 호출할 때 마다 시간을 측정해서 실행에 소요된 시간, 전달된 인수, 반환값을 출력하는 데코레이터를 만들어보자

In [33]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [34]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12803154s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000033s] factorial(1) -> 1
[0.00000642s] factorial(2) -> 2
[0.00001054s] factorial(3) -> 6
[0.00001379s] factorial(4) -> 24
[0.00001792s] factorial(5) -> 120
[0.00002171s] factorial(6) -> 720
6! = 720


In [37]:
# 위의 코드는 아래코드와 같다.
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [53]:
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)
factorial = clock(factorial)
factorial.__name__

'clocked'

위에서 구현한 clock()데커레이터는 단점이 몇 가지 있다. 키워드 인수를 지원하지 않으며, 데커레이트된 함수의 ___name__과 __doc__속성을 가린다. functools.wraps 데커레이터를 사용하면 이런 문제를 해결할 수 있다.

In [52]:
import functools

def clock2(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []

        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))

        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result

## 7.8 표준 라이브러리에서 제공하는 데코레이터
파이썬에서는 기본 데코레이터로 사용할 수 있는 property(), classmethod(), staticmethod()같은 내장함수를 제공한다. 다 나중에 설명한다고 합니다.
자주 볼 수 있는 functools.wraps()가 있다. 이 함수는 제대로 작동하는 데코레이터를 만들기 위한 것이다. 표준 라이브러리가 제공하는 데코레이터들 중 lru_cache()와 singledispatch()는 흥미롭다. lru_cache()는 함수의 반환값을 메모이제이션해서 이전에 호출한 인수에 대한 결과를 빠르게 반환한다. singledispatch()는 함수를 여러 구현으로 다형성을 제공한다.

### 7.8.1 functools.lru_cache()를 이용한 메모이제이션
functools.lru_cache()는 실제로 쓸모가 많은 데코레이터로, 메모이제이션(memoization)을 구현한다. 메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다. 이름 앞에 붙은 LRU(least recently used)는 메모이제이션에 사용된 항목을 제거할 때 사용된다. 이 데코레이터는 인수가 해시 가능해야 한다. 그렇지 않으면 TypeError가 발생한다.
아래예시에서 처럼 n번째 피보나치 수열을 생성하기 위해 아주 느리게 실행되는 재귀 함수에서 lru_cache() 데코레이터가 진가를 발휘한다.

In [54]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000071s] fibonacci(0) -> 0
[0.00000100s] fibonacci(1) -> 1
[0.00188829s] fibonacci(2) -> 1
[0.00000058s] fibonacci(1) -> 1
[0.00000071s] fibonacci(0) -> 0
[0.00000058s] fibonacci(1) -> 1
[0.00002183s] fibonacci(2) -> 1
[0.00004329s] fibonacci(3) -> 2
[0.00195287s] fibonacci(4) -> 3
[0.00000054s] fibonacci(1) -> 1
[0.00000058s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00002125s] fibonacci(2) -> 1
[0.00004187s] fibonacci(3) -> 2
[0.00000050s] fibonacci(0) -> 0
[0.00000054s] fibonacci(1) -> 1
[0.00002087s] fibonacci(2) -> 1
[0.00000058s] fibonacci(1) -> 1
[0.00000058s] fibonacci(0) -> 0
[0.00000054s] fibonacci(1) -> 1
[0.00002129s] fibonacci(2) -> 1
[0.00004192s] fibonacci(3) -> 2
[0.00008258s] fibonacci(4) -> 3
[0.00014454s] fibonacci(5) -> 5
[0.00213025s] fibonacci(6) -> 8
8


In [65]:
# 캐시를 이용해 더 빠르게 구현합니다.

@functools.lru_cache()

@clock      # 오 이런식으로 데코레이터를 떨어뜨려놔도 되는건지요?


def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000075s] fibonacci(0) -> 0
[0.00000096s] fibonacci(1) -> 1
[0.00009625s] fibonacci(2) -> 1
[0.00000108s] fibonacci(3) -> 2
[0.00012008s] fibonacci(4) -> 3
[0.00000083s] fibonacci(5) -> 5
[0.00015083s] fibonacci(6) -> 8
8


In [66]:
functools.lru_cache(maxsize=128, typed=False)
"""
maxsize: 저장할 아규먼트의 개수
typed: 아규먼트의 자료형을 구분할지 여부
"""

'\nmaxsize: 저장할 아규먼트의 개수\ntyped: 아규먼트의 자료형을 구분할지 여부\n'

### 7.8.2 단일 디스패치를 이용한 범용 함수
웹 어플리케이션을 디버깅하는 도구를 만들고 있다. 파이썬 객체의 자료형마다 HTML코드를 생성하고자 한다.

In [67]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [69]:
print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co.\n- a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


singledpatch() 데코레이터로 범용 함수를 만들어주자

In [71]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'


In [72]:
print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co.<br>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.9 누적된 데코레이터
데코레이터를 여러 개 적용할 수 있다. 데코레이터를 적용하면 함수가 데코레이터로 감싸지고, 그 결과는 다른 함수로 감싸진다. 이런식으로 데코레이터를 적용하면 데코레이터가 적용된 함수는 데코레이터가 적용된 순서의 역순으로 실행된다.
```python
@decorator1
@decorator2
def func():
    pass
```
위 코드는 아래 코드와 같다.
```python
def func():
    pass
func = decorator1(decorator2(func))
```

## 7.10 매개변수화된 데코레이터
데코레이터를 파싱할 때 파이썬은 데코레이트된 함수를 가져와서 데코레이터에 인수로 넘겨준다. 그러면 어떻게 다른 인수를 받는 데코레이터를 만들 수 있을까? 설명이 복잡하다고 하니 예시로 살펴보자

In [78]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)

running register(<function f1 at 0x107e1ee50>)
running main()
registry -> [<function f1 at 0x107e1ee50>]


### 7.10.1 매개변수화된 등록 데코레이터
register가 등록하는 함수를 활성화 혹은 비활성화하기 쉽게 만들기 위해, 선택적인 인수 active를 받도록 만들어보자. active가 False면 데코레이트된 함수를 등록해제한다. 새로 만든 register()함수는 개념적으로 데코레이터가 아니라 데코레이터 팩토리다. 호출되면 반환하기 때문이다.

In [82]:
registry = set()

def register(active=True):
    def decorate(func):
        print(f'running register(active={active}) -> decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')


^C
running register(active=False) -> decorate(<function f1 at 0x107e7fca0>)
running register(active=True) -> decorate(<function f2 at 0x107e7fe50>)


In [87]:
import Ch7.registration_param as registration_param

registration_param.registry     # f2만 registry에 남아 있음에 주의하라. f1에는 active=False가 지정되어 있기 때문이다.

{<function Ch7.registration_param.f2()>}

In [94]:
from Ch7.registration_param import register, registry

print(register,end='\n\n')
print(register()(f3),end='\n\n')
print(registry ,end='\n\n')
print(register(active=False)(f2),end='\n\n')
print(registry)

<function register at 0x107e7fa60>

running register(active=True) -> decorate(<function f3 at 0x107e1fa60>)
<function f3 at 0x107e1fa60>

{<function f2 at 0x107e87310>, <function f3 at 0x107e1fa60>}

running register(active=False) -> decorate(<function f2 at 0x107e7fe50>)
<function f2 at 0x107e7fe50>

{<function f2 at 0x107e87310>, <function f3 at 0x107e1fa60>}


### 7.10.2 매개변수화된 clock 데코레이터

In [95]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12802505s] snooze(0.123) -> None
[0.12804580s] snooze(0.123) -> None
[0.12802005s] snooze(0.123) -> None


In [96]:
@clock('{name}: {elapsed}s')    # fmt 인수를 지정
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12669587135314941s
snooze: 0.12801623344421387s
snooze: 0.12447595596313477s


In [97]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s
snooze(0.123) dt=0.126s
